<a href="https://colab.research.google.com/github/unconst/GradientBidding/blob/master/PriceOfAnarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Price of Anarchy

In [105]:
import tensorflow as tf
import types
import random
import numpy
from numpy import linalg as LA
numpy.set_printoptions(precision=3, suppress=True)


In [140]:
def alloc(mode, contrib, hparams):
    graph = tf.Graph()
    session = tf.Session(graph=graph)
    
    # Build optimization problem.
    with graph.as_default():
        
        # Contribution: W* :  The True (i,j)-utility  
        Contribution = tf.constant(contrib, tf.float32)

        # Weights: W: Contribution row of W.
        weights_list = []
        for _ in range(n):
            weights_i = tf.Variable(tf.random.uniform((1, hparams.n_nodes), minval=0))
            weights_list.append(weights_i)
        Weights = tf.concat(weights_list, axis=0)
        
        # We force weights onto [0,1] using the sigmoid and then normalize.
        Weights_sig = tf.sigmoid(Weights)
        Weights_norm = tf.linalg.normalize(Weights_sig, ord=1, axis=1)[0]
        
        # Weights_diag: Is the matrix with the main diagonal of Weights. a.k.a self-contribution.
        Weights_diag = tf.matrix_set_diag(tf.zeros((hparams.n_nodes, hparams.n_nodes)), tf.linalg.tensor_diag_part(Weights_norm), k = 0)

        # Interranking: Q: The inter-model ranking derived from the weights. We use the infinite series
        # absorbing markov chain calculation. 
        Interranking = tf.linalg.inv(tf.eye(hparams.n_nodes) - Weights_norm + Weights_diag)
        Interranking = tf.linalg.normalize(Interranking, ord=1, axis=1)[0]

        # Divergence score.
        Interranking_mean = tf.reshape(tf.tile(tf.reduce_mean(Interranking, axis=0), [hparams.n_nodes]), [hparams.n_nodes, hparams.n_nodes])
        cross_entropy = -tf.reduce_sum(tf.multiply(Interranking_mean, tf.log(Interranking)), axis=1)
        Divergence = hparams.gamma * tf.reshape(cross_entropy, [10])
        
        # Emission: E :The quantity of additional stake attributed to the nodes.
        Emission = tf.linalg.tensor_diag_part(Interranking * Weights_diag)

        # Market_Contribution: The allocation function shifted contribution scores given the bids.
        Market_Shift = tf.reduce_mean(Weights_norm, axis=1)
        Masked_Contribution = tf.multiply(Contribution, tf.nn.relu(Weights_norm - Market_Shift))
        
        # Utility: U: The utility gained from the loss function given the masked contributions.
        Utility = tf.reduce_sum(Masked_Contribution, axis=1)

        # Payoff: P: Utility + Emission
        Payoff = Utility + Emission - Divergence
        
        ### Bellow Optimization.

        # Bidders move in the direction of the gradient of the Payoff.
        optimizer = tf.train.AdamOptimizer(hparams.learning_rate)
        
        # Mode == Competitive: All nodes optimize only their local payoff. 
        train_steps = []
        if mode == 'competitive':
            for i in range(hparams.n_nodes):
                payoff_i = tf.slice(Payoff, [i], [1])
                weights_i = weights_list[i]
                grads_and_vars_i = optimizer.compute_gradients(loss=-payoff_i, var_list=[weights_i])
                train_steps.append(optimizer.apply_gradients(grads_and_vars_i))

        # Mode == Coordinated: Coordinated nodes optimize the aggregated payoff
        elif mode == 'coordinated':
            grads_and_vars = optimizer.compute_gradients(loss=-tf.reduce_mean(Payoff), var_list=weights_list)
            train_steps.append(optimizer.apply_gradients(grads_and_vars))

        # Init the graph.
        session.run(tf.global_variables_initializer())

        # Converge...
        for step in range(hparams.n_steps):
            
            # Randomly choose participant to optimize
            if mode == 'competitive':
                step = random.choice(train_steps)
                
            # Optimize all participants.
            elif mode == 'coordinated':
                step = train_steps[0]
                
            # Run graph.
            output = session.run(fetches = 
                                      {   
                                        'step': step,       
                                        'Payoff': Payoff,   
                                        'Utility': Utility, 
                                        'Emission': Emission,
                                        'Divergence': Divergence,
                                        'Weights': Weights_norm,  
                                        'Mask': Masked_Contribution,
                                        'Interranking': Interranking,
                                      })      
        # Return metrics.
        return output


In [146]:
hparams = types.SimpleNamespace( 
    trials = 1,
    n_steps = 1000,
    learning_rate = 0.05,
    n_nodes = 10,
    contrib_factor = 1,
    gamma=0.5
)


for _ in range(hparams.trials):
    
    # Build "True" contribution matrix.
    contribution = numpy.random.randn(hparams.n_nodes, hparams.n_nodes)
    contribution = (contribution - numpy.min(contribution))/numpy.ptp(contribution)
    contribution = contribution/contribution.sum(axis=1, keepdims=1) * hparams.contrib_factor
    
    print ('Contribution: W*')
    print (contribution)
    print ('')
    
    # Run coordinated weight convergence.
    coord_output = alloc('coordinated', contribution, hparams)
    
    # Run competitive weight convergence.
    comp_output = alloc('competitive', contribution, hparams)
    
    print ('Coordinated Mask: M')
    print (coord_output['Mask'])
    print ('')
    
    print ('Competitive Mask: M')
    print (comp_output['Mask'])
    print ('')

    print ('Coordinated Weights: W')
    print (coord_output['Weights'])
    print ('')
    
    print ('Competitive Weights: W')
    print (comp_output['Weights'])
    print ('')
    
    print ('Coordinated Interranking: Q')
    print (coord_output['Interranking'])
    print ('')
    
    print ('Competitve Interranking: Q')
    print (comp_output['Interranking'])
    print ('')
    
    print ('Coordinated Divergence: D')
    print (coord_output['Divergence'])
    print ('')
    
    print ('Competitive Divergence: D')
    print (comp_output['Divergence'])
    print ('')
    
    print ('Coordinated Emission: E')
    print (coord_output['Emission'])
    print ('')
    
    print ('Competitive Emission: E')
    print (comp_output['Emission'])
    print ('')
    
    print ('Coordinated Utility: U')
    print (coord_output['Utility'])
    print ('')
    
    print ('Competitive Utility: U')
    print (comp_output['Utility'])
    print ('')
        
    print ('Coordinated Payoff: P')
    print (coord_output['Payoff'])
    print ('Avg:' + str(sum(coord_output['Payoff'])/hparams.n_nodes))
    print ('')
    
    print ('Competitive Payoff: P')
    print (comp_output['Payoff'])
    print ('Avg:' + str(sum(comp_output['Payoff'])/hparams.n_nodes))
    print ('')

    poa = sum(comp_output['Payoff']) / sum(coord_output['Payoff'])
    print ('Price of Anarchy: ' + str(poa))



Contribution: W*
[[0.099 0.118 0.094 0.138 0.11  0.068 0.176 0.087 0.029 0.082]
 [0.14  0.1   0.147 0.1   0.091 0.077 0.091 0.047 0.135 0.074]
 [0.102 0.114 0.106 0.11  0.064 0.111 0.131 0.066 0.071 0.124]
 [0.113 0.091 0.071 0.132 0.088 0.065 0.125 0.144 0.056 0.116]
 [0.073 0.06  0.134 0.127 0.043 0.08  0.161 0.127 0.091 0.105]
 [0.15  0.099 0.1   0.086 0.082 0.072 0.094 0.107 0.069 0.142]
 [0.13  0.083 0.152 0.129 0.07  0.095 0.09  0.044 0.084 0.121]
 [0.121 0.123 0.081 0.064 0.119 0.131 0.05  0.082 0.126 0.103]
 [0.08  0.105 0.091 0.145 0.111 0.153 0.071 0.065 0.045 0.133]
 [0.122 0.086 0.098 0.076 0.156 0.135 0.    0.079 0.107 0.141]]

Coordinated Mask: M
[[0.    0.    0.    0.    0.    0.    0.159 0.    0.    0.   ]
 [0.125 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.117 0.    0.    0.   ]
 [0.045 0.    0.    0.    0.    0.    0.05  0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.145 0.    0.    0.   ]
 [0.065 0.    0.